In [1]:
!pip install transformers

### Pretrained BERT Model Load

In [2]:
import torch
from transformers import AutoModel, AutoTokenizer

In [3]:
MODEL_NAME = "bert-base-multilingual-cased"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModel.from_pretrained(MODEL_NAME)
model.parameters

<bound method Module.parameters of BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(119547, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Drop

In [4]:
!git clone https://github.com/songys/Chatbot_data.git

fatal: destination path 'Chatbot_data' already exists and is not an empty directory.


In [5]:
import pandas as pd

data = pd.read_csv('Chatbot_data/ChatbotData .csv')

In [6]:
data.head()

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0


In [7]:
data.label

0        0
1        0
2        0
3        0
4        0
        ..
11818    2
11819    2
11820    2
11821    2
11822    2
Name: label, Length: 11823, dtype: int64

In [8]:
chatbot_Question = data['Q'].to_list()
chatbot_Answer = data['A'].to_list()

### [CLS] token을 얻기 위한 함수

In [9]:
def get_cls_token(sent_A):
    model.eval()
    tokenized_sent = tokenizer(
            sent_A,
            return_tensors="pt",
            truncation=True,
            add_special_tokens=True,
            max_length=128
    )
    with torch.no_grad():# 그라디엔트 계산 비활성화
        outputs = model(    # **tokenized_sent
            input_ids=tokenized_sent['input_ids'],
            attention_mask=tokenized_sent['attention_mask'],
            token_type_ids=tokenized_sent['token_type_ids']
            )
    logits = outputs.last_hidden_state[:,0,:].detach().cpu().numpy()
    return logits

### Chatbot 데이터셋의 질문 [CLS] token 확인

In [10]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

dataset_cls_hidden = []
for q in chatbot_Question:
    q_cls = get_cls_token(q)
    dataset_cls_hidden.append(q_cls)
dataset_cls_hidden = np.array(dataset_cls_hidden).squeeze(axis=1)
print(dataset_cls_hidden)   # 데이터셋의 질문에 대한 [CLS] 토큰 벡터
print(dataset_cls_hidden.shape)


[[-9.8364174e-02 -2.8734552e-02 -6.7383128e-01 ...  7.0865905e-01
   6.9995590e-02  3.6694250e-01]
 [ 6.4907528e-02  7.1816310e-02 -1.4097233e-01 ...  1.3833308e-01
   1.6595210e-01  1.1292047e-01]
 [ 4.0807392e-04  2.2837043e-02  2.5835195e-01 ...  1.3542069e-01
   2.6734051e-01  9.5744975e-02]
 ...
 [ 1.5377924e-01 -2.8241105e-02 -8.1072405e-02 ...  2.2878532e-01
   1.6532596e-02  1.2961590e-01]
 [-4.5103926e-02  1.0422598e-01 -1.8926837e-01 ...  5.9134108e-01
   2.6985615e-01 -2.9995786e-02]
 [ 3.5281718e-02  3.9880634e-03  1.6102602e-01 ...  6.0296750e-01
   4.5698886e-03  4.1516870e-01]]
(11823, 768)


### Cosine Similarity

In [11]:
query = '날씨 좋다.'
query_cls_hidden = get_cls_token(query)
print(query_cls_hidden)
print(query_cls_hidden.shape)

[[ 3.83281946e-01 -1.76761881e-01  3.25298280e-01  2.42025852e-01
  -1.31881265e-02  1.18152499e-01 -2.57624179e-01  8.45876485e-02
   5.18225551e-01  1.13024652e-01  6.06042445e-02  2.61006206e-01
  -5.52728236e-01  1.02738731e-01 -5.72915792e-01 -1.80312302e-02
   1.13426298e-01  4.76545811e-01  5.10438047e-02 -2.46569708e-01
   2.41846666e-01  1.57776758e-01 -4.17527467e-01  3.90358597e-01
  -1.55721484e-02  4.23006713e-01  7.83118904e-02 -1.02301817e-02
   7.29462087e-01 -9.92010087e-02  4.73538078e-02 -3.54308933e-01
  -2.42681727e-01  1.15566202e-01 -1.25359334e-02  2.27487370e-01
  -1.49591136e+00 -2.15621382e-01  1.57757681e-02 -1.23456039e-01
  -3.73227417e-01  5.29242009e-02 -1.80557892e-01  2.24892020e-01
  -6.72061518e-02  1.38467515e+00  3.43385935e-01 -1.71069175e-01
   1.29491627e+00 -1.75853774e-01 -8.32180381e-02 -1.26145452e-01
   1.10069014e-01 -1.63968539e+00 -1.50627449e-01  4.39143330e-01
   3.69856469e-02 -3.23789924e-01 -1.47149548e-01  1.15671039e-01
  -2.04593

In [12]:
cos_sim = cosine_similarity(query_cls_hidden, dataset_cls_hidden)
print(cos_sim)

[[0.5531573  0.8173564  0.8368767  ... 0.8463146  0.83902025 0.8253838 ]]


In [13]:
top_question = np.argmax(cos_sim)

print('나의 질문: ', query)
print('저장된 답변: ', chatbot_Answer[top_question])

나의 질문:  날씨 좋다.
저장된 답변:  멋져 보여요!


In [14]:
query = '어? 이쁘다.'
query_cls_hidden = get_cls_token(query)
print(query_cls_hidden)
print(query_cls_hidden.shape)

[[ 1.11384675e-01 -5.58543205e-02  1.29372805e-01  4.65893865e-01
  -2.30073303e-01  1.81989998e-01 -2.45560244e-01 -5.21942526e-02
   2.24047303e-01  4.82744128e-01  6.10248819e-02  1.08832866e-01
   1.26260370e-01  6.88655525e-02 -7.21856654e-01 -7.94888139e-02
  -1.00030795e-01  4.77906287e-01 -2.19027996e-01  1.43951386e-01
   3.58569026e-01  1.68266118e-01  5.81273213e-02  2.11213708e-01
  -3.32390182e-02  2.70364076e-01  3.68893772e-01  3.75857502e-01
   6.83013201e-01  2.73676187e-01 -4.79547074e-03 -1.16815187e-01
  -1.85883433e-01  9.05937105e-02 -1.12510674e-01  7.63782635e-02
  -2.26979017e+00  1.12055941e-02 -5.01202121e-02 -1.25131994e-01
  -3.03072721e-01 -3.70272458e-01 -4.90571596e-02  6.37527332e-02
  -1.63679232e-03  1.15429235e+00  3.49326789e-01  7.59086311e-02
   1.37570834e+00 -3.96482587e-01  8.15963745e-02 -4.95552719e-01
  -9.35000554e-02 -1.55707121e+00  2.16919079e-01 -6.82606772e-02
  -8.41042399e-02 -5.35420962e-02  3.70276496e-02 -3.06223333e-02
  -1.41020

In [15]:
cos_sim = cosine_similarity(query_cls_hidden, dataset_cls_hidden)
print(cos_sim)

[[0.54317635 0.91408527 0.9261826  ... 0.93729025 0.94570184 0.8951477 ]]


In [16]:
top_question = np.argmax(cos_sim)

print('나의 질문: ', query)
print('저장된 답변: ', chatbot_Answer[top_question])

나의 질문:  어? 이쁘다.
저장된 답변:  빼면 깔끔해질 거예요.


In [17]:
query = '성능 잘 나오기로 약속~'
query_cls_hidden = get_cls_token(query)
print(query_cls_hidden)
print(query_cls_hidden.shape)

[[ 4.07076590e-02 -1.34148270e-01 -1.48813531e-01  2.50741187e-02
  -7.46419206e-02  2.50209183e-01  2.16360196e-01  1.17053270e-01
  -4.30584773e-02  3.24991047e-01 -7.46682808e-02  1.44304797e-01
  -2.84400880e-02  2.49946397e-02 -4.60282534e-01  7.64180869e-02
  -2.38289803e-01  1.46765292e-01  2.55973309e-01  1.98114902e-01
   3.14339727e-01  4.41972427e-02  2.59205811e-02  2.13649035e-01
  -1.81956440e-02 -8.05270970e-02  1.51534632e-01  5.09416424e-02
   4.36880350e-01  2.31057748e-01  1.93872407e-01  4.57580015e-02
  -4.77560520e-01  1.32565916e-01 -8.37435666e-03 -5.78137068e-03
  -1.69378638e+00  1.78969175e-01 -2.48629358e-02  1.27469197e-01
  -2.32573956e-01  3.72404993e-01 -2.08590448e-01  1.14213072e-01
   1.82512879e-01  1.22814250e+00  3.02286953e-01  2.78338462e-01
   1.14296854e+00 -3.35205764e-01  2.37277567e-01 -5.41359961e-01
  -1.83206052e-03 -1.48592079e+00  3.52902502e-01  7.64993671e-03
   3.54178529e-03 -2.77434975e-01 -1.54502928e-01  8.11427832e-02
  -3.31738

In [18]:
cos_sim = cosine_similarity(query_cls_hidden, dataset_cls_hidden)
print(cos_sim)

[[0.5516609  0.8927808  0.91096133 ... 0.89150715 0.89623326 0.8831834 ]]


In [19]:
top_question = np.argmax(cos_sim)

print('나의 질문: ', query)
print('저장된 답변: ', chatbot_Answer[top_question])

나의 질문:  성능 잘 나오기로 약속~
저장된 답변:  글 쓰면서 정리가 되기도 해요.


In [20]:
query = '여행 가고 싶다~'
query_cls_hidden = get_cls_token(query)
print(query_cls_hidden)
print(query_cls_hidden.shape)

[[ 1.66688696e-01 -8.17565769e-02  2.44899213e-01 -4.11414914e-02
   3.00019532e-01  3.25544387e-01  1.07955284e-01 -6.30285889e-02
  -8.74464437e-02  2.76034951e-01 -7.70686381e-03  3.79292756e-01
  -1.93564013e-01  1.84937418e-01 -1.34110183e-01 -1.10461324e-01
  -1.74152926e-01  3.86260711e-02  4.40778673e-01 -1.13269068e-01
   2.99505264e-01  3.62773687e-02 -2.65274756e-03  1.39368623e-01
  -7.78839141e-02  1.15071371e-01  4.07707542e-01 -1.40264943e-01
   5.48159778e-01  3.45284969e-01  4.08029407e-01  4.31272648e-02
  -2.79395550e-01  2.85395980e-01  3.13143790e-01  2.48671904e-01
  -1.72903693e+00  1.09242514e-01 -1.03410959e-01 -4.77860086e-02
  -4.94920313e-01  2.65665740e-01  2.93148279e-01  8.06198567e-02
   4.70977306e-01  1.08404958e+00  5.92423797e-01  1.14442840e-01
   1.50949574e+00 -4.86019641e-01  2.86386728e-01 -4.04011369e-01
  -7.82305934e-03 -1.59030712e+00  1.58661589e-01  3.08110774e-01
   3.15409482e-01  2.64069736e-02  9.00326818e-02 -4.73908745e-02
  -2.57003

In [21]:
cos_sim = cosine_similarity(query_cls_hidden, dataset_cls_hidden)
print(cos_sim)

[[0.5316211  0.7993492  0.8379463  ... 0.78359383 0.81474483 0.887994  ]]


In [22]:
top_question = np.argmax(cos_sim)

print('나의 질문: ', query)
print('저장된 답변: ', chatbot_Answer[top_question])

나의 질문:  여행 가고 싶다~
저장된 답변:  좋은 만남이었길 바랍니다.


In [24]:
query = '제 이름 외자에요. 외자라서 외로움을 많이 타죠'
query_cls_hidden = get_cls_token(query)
print(query_cls_hidden)
print(query_cls_hidden.shape)

[[ 6.75736219e-02  1.83811814e-01 -2.46047080e-01  2.90474176e-01
  -1.57525092e-01  6.02027066e-02 -1.14208415e-01 -3.40961069e-02
   1.27469808e-01  3.26148033e-01 -9.13114250e-02 -1.25408053e-01
  -1.89538505e-02  2.00252324e-01 -4.95044172e-01 -2.00359091e-01
  -9.87146646e-02  2.83449829e-01  7.85427615e-02  2.17943057e-01
   2.41445675e-01  1.14528388e-01 -7.69752562e-02  7.52804950e-02
   8.06967840e-02 -3.06557375e-03  5.12247384e-01  6.10091090e-02
   7.47031331e-01  3.53354782e-01  3.46629657e-02  7.62205431e-03
  -1.44173995e-01  9.23534334e-02  1.06674932e-01  2.39246130e-01
  -2.14033580e+00 -1.24233231e-01 -2.81160399e-02  5.21288738e-02
  -4.78978068e-01 -2.99783722e-02  1.80604942e-02  1.49943858e-01
   1.87861651e-01  1.25977278e+00  3.78004104e-01 -5.53029776e-02
   1.32657623e+00 -3.81917000e-01  1.07243411e-01 -6.81689799e-01
   2.29058675e-02 -1.67979050e+00  1.75104544e-01  1.23587497e-01
   3.45879458e-02  1.55676818e-02  1.88760921e-01 -9.11315624e-03
  -7.67123

In [25]:
cos_sim = cosine_similarity(query_cls_hidden, dataset_cls_hidden)
print(cos_sim)

[[0.51788014 0.90697575 0.912522   ... 0.92149234 0.92753756 0.9167131 ]]


In [26]:
top_question = np.argmax(cos_sim)

print('나의 질문: ', query)
print('저장된 답변: ', chatbot_Answer[top_question])

나의 질문:  제 이름 외자에요. 외자라서 외로움을 많이 타죠
저장된 답변:  상황이 여의치 않나봐요.
